# Importing Libraries

In [3]:
!pip install --upgrade tensorflow --quiet
!pip install matplotlib --quiet
!pip install opencv-python 

In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Loading Images

In [5]:
# Set up paths
data_dir = 'Corals' 
img_height, img_width = 256, 256  
batch_size = 32  

# Create an ImageDataGenerator for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,  
    validation_split=0.2  # Use 20% of data for validation
)

# Load training data
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary', 
    subset='training'  # Set as training data
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  
    subset='validation'  # Set as validation data
)

Found 8976 images belonging to 2 classes.
Found 2243 images belonging to 2 classes.


# Define the Model

In [ ]:
def generate_fixed_proposals(image, num_proposals=5, window_size=(256, 256), step_size=50):
    proposals = []
    for y in range(0, image.shape[0] - window_size[1] + 1, step_size):
        for x in range(0, image.shape[1] - window_size[0] + 1, step_size):
            window = image[y:y + window_size[1], x:x + window_size[0], :]
            if window.shape == (256, 256, 3):
                proposals.append(window)
            if len(proposals) == num_proposals:
                break
        if len(proposals) == num_proposals:
            break
            
    # If not enough proposals, pad with zeros
    while len(proposals) < num_proposals:
        proposals.append(np.zeros(window_size + (3,)))
        
    return np.array(proposals)

# Evaluate the model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# Define the input shape
input_shape = (256, 256, 3)

# Load VGG16 as the backbone for feature extraction
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Define additional layers on top of the base model for RCNN
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
output = Dense(2, activation='softmax')(x)  
# Create the final RCNN model using VGG16 as the backbone
rcnn_model = Model(inputs=base_model.input, outputs=output)

rcnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Custom Data Generator
class ProposalDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, generator, num_proposals=5):
        self.generator = generator
        self.num_proposals = num_proposals

    def __len__(self):
        return len(self.generator)

    def __getitem__(self, index):
        batch_data, batch_labels = self.generator[index]
        proposals = []
        for img in batch_data:
            props = generate_fixed_proposals(img, self.num_proposals)
            proposals.append(props)

        proposals = np.array(proposals)  # Shape: (batch_size, num_proposals, height, width, channels)
        proposals = proposals.reshape(-1, 256, 256, 3)  # Shape: (batch_size * num_proposals, 256, 256, 3)

        # Repeat labels for each proposal
        batch_labels = np.repeat(batch_labels, self.num_proposals, axis=0)  # Shape: (batch_size * num_proposals, 1)
        batch_labels = to_categorical(batch_labels, num_classes=2)  # Convert to one-hot encoding

        return proposals, batch_labels

# Assuming `train_generator` is already defined
# Create custom data generator
custom_generator = ProposalDataGenerator(train_generator, num_proposals=5)

# Train the RCNN model
rcnn_model.fit(custom_generator, epochs=20, verbose=1)

In [ ]:
loss, accuracy = rcnn_model.evaluate(test_generator)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


# Saving the model Images

In [2]:
rcnn_model.save('complex_rcnn_model.h5')